## Функция Cosine similarity

In [1]:
def cosine_similarity(x_predict, x):
    if type(x_predict) is np.ndarray:
        flat_output = x_predict
        flat_input = x_predict
        # flat_output = np.reshape(x_predict, (np.shape(x)[0], -1))
        # flat_input = np.reshape(x_predict, (np.shape(x)[0], -1))
        sum = np.sum(flat_output * flat_input, -1)
        norm1 = np.linalg.norm(flat_output, axis=-1) + 0.000001
        norm2 = np.linalg.norm(flat_input, axis=-1) + 0.000001 
        return -(sum / norm1 / norm2)
    else:
        # ДЛЯ НЕ ПОЛНОСВЯЗНЫХ СЛОЕВ НУЖЕН ДРУГОЙ shape
        flat_output = x_predict
        flat_input = x_predict
        # flat_output = tf.reshape(tensor=x_predict, shape=[x.shape.as_list()[0], -1])
        # flat_input = tf.reshape(tensor=x_predict, shape=[x.shape.as_list()[0], -1])
        sum = tf.math.reduce_sum(tf.math.multiply(flat_output, flat_input), axis=-1)
        norm1 = tf.norm(flat_output, axis=-1) + 0.000001
        norm2 = tf.norm(flat_input, axis=-1) + 0.000001
        return -(tf.math.divide(tf.math.divide(sum, norm1), norm2))

# Universal Sentence Encoder

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
hub_layer = hub.KerasLayer(
    'https://tfhub.dev/google/universal-sentence-encoder/4',
    input_shape=[], 
    dtype=tf.string,
    trainable=True)

## RSRAE model

In [3]:
import math
import numpy as np
print("Tensorflow version = {}".format(tf.__version__)) # текущая версия tf

from tensorflow.keras import Model, optimizers, metrics
from tensorflow.keras.layers import Layer, Flatten, Dense, BatchNormalization, Dropout

# from tensorflow.keras import activations, Sequential, Input
# from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Reshape
# from sklearn.metrics import roc_auc_score, average_precision_score

from sklearn.metrics import roc_auc_score, average_precision_score

# Задаем random_seed для tensorflow и numpy
random_seed = 123
tf.random.set_seed(random_seed)
np.random.seed(random_seed)

# Sets the default float type
tf.keras.backend.set_floatx('float64')

# Set random seed
tf.random.set_seed(123)
np.random.seed(123)


class RSR(Layer):
    """
    Robust Subspace Recovery (RSR) layer.
    Робастный слой, восстанавливающий подпространство. Задача данного слоя - отобразить
    закодированные энкодером данные в подпростраство так, чтобы после их обратного
    отображения декодером дивергенция между экземпляром исходных данных и его образом,
    полученным от автоэнкодера была незначительной для нормального экземпляра и была
    большой для аномального экземпляра. 

    # Example
    ```
        z_rsr, A = RSR(intrinsic_size=10)(z)
    ```
    # Arguments
        intrinsic_size: размерность z_rsr.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)` after encoding.
    # Output shape
        2D tensor with shape: `(n_samples, intrinsic_size)`.
    """

    def __init__(self, intrinsic_size: int, name="RSR_layer", **kwargs):
        super(RSR, self).__init__(name=name, **kwargs)
        # Если присваивать экземпляр слоя, как атрибут другого слоя, то хорошей
        # практикой делать создавать такие подслои в __init__ (поскольку подслои обычно
        # имеют метод build, они будут собраны, когда будет собран внешний слой). 
        self.flatten = Flatten()
        self.intrinsic_size = intrinsic_size
        
    def build(self, input_shape):
        """Определяет веса слоя, а именно задает матрицу A."""
        self.A = self.add_weight(name="A",
                                 shape=[int(input_shape[-1]), self.intrinsic_size],
                                 initializer='random_normal',
                                 trainable=True,)
        
        # self.V = self.add_weight(name="V",
        #                          shape=[int(input_shape[-1]), 1],
        #                          initializer='random_normal',
        #                          trainable=True,)
        
    def call(self, z):
        """
        Логика слоя. Умножение выхода энкодера - вектора z на матрицу A.
        Возвращает отображенный z_rsr и матрицу A, которая потребуется далее.
        """
        z = self.flatten(z)
        # print("z.shpae Before A:", z.shape)
        z_rsr = tf.linalg.matmul(z, self.A)
        # print("z_rsr.shpae After A:", z_rsr.shape) 
        return z_rsr

    # Опционально, пользовательский слой может быть сериализован реализацией метода 
    # get_config и метода класса (@classmethod) from_config.
    def get_config(self):
        config = super(Layer, self).get_config()
        config.update({'intrinsic_size': self.intrinsic_size})
        return config

    # На самом деле нет необходимости определять `from_config` здесь, поскольку 
    # возвращение `cls(**config)` - поведение по умолчанию.
    @classmethod
    def from_config(cls, config):
        return cls(**config)


class L2Normalization(Layer):
    """Слой для l_2 нормализации, который будет применяться к выходу RSR layer."""
    
    def __init__(self, name="L2Normalization", **kwargs):
        super(L2Normalization, self).__init__(name=name, **kwargs)

    def call(self, z_rsr):
        """
        Выполняет l_2 нормализацию векторов, полученных после применения RSR layer
        вдоль оси, соответсвующей числу признаков. То есть производится нормализация
        каждого экземпляра выборки, в результате которой признаки экземпляров будут
        находиться в отрезке [-1; 1].
        """
        z_tilde = tf.math.l2_normalize(z_rsr, axis=-1)
        return z_tilde

    # Опционально, пользовательский слой может быть сериализован реализацией метода 
    # get_config и метода класса (@classmethod) from_config.
    def get_config(self):
        config = super(Layer, self).get_config()
        return config

    # На самом деле нет необходимости определять `from_config` здесь, поскольку 
    # возвращение `cls(**config)` - поведение по умолчанию.
    @classmethod
    def from_config(cls, config):
        return cls(**config)


class Encoder(Layer):
    """
    Класс для encoder модели RSRAE. Отображает исходные данные input_data в вектор z,
    кодирующий исходные данные.
    """

    def __init__(self,
                 hidden_layer_dimensions,
                 activation,
                 flag_bn=True, 
                 name="Encoder",
                 **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.hidden_layer_dimensions = hidden_layer_dimensions
        self.activation = activation
        self.flag_bn = flag_bn
        self.dense0 = Dense(hidden_layer_dimensions[0], activation=activation,
                            name='encoder_0')
        self.dense1 = Dense(hidden_layer_dimensions[1], activation=activation,
                            name='encoder_1')
        self.dense2 = Dense(hidden_layer_dimensions[2], activation=activation,
                            name='encoder_2')
        if flag_bn:
            self.batch_normalization0 = BatchNormalization(name="encoder_bn_layer_0")
            self.batch_normalization1 = BatchNormalization(name="encoder_bn_layer_1")
            self.batch_normalization2 = BatchNormalization(name="encoder_bn_layer_2")

    def call(self, inputs):
        """Отображние исходных данных x -> в закодированный вектор z."""
        x = inputs
        x = self.dense0(x)
        if self.flag_bn:
            x = self.batch_normalization0(x)
        x = self.dense1(x)
        if self.flag_bn:
            x = self.batch_normalization1(x)
        x = Dropout(0.2)((x))
        x = self.dense2(x)
        if self.flag_bn:
            x = self.batch_normalization2(x)
        x = Dropout(0.2)((x))    
        z = x
        return z
    
    # Опционально, пользовательский слой может быть сериализован реализацией метода 
    # get_config и метода класса (@classmethod) from_config.
    def get_config(self):
        config = super(Layer, self).get_config()
        config.update({'hidden_layer_dimensions': self.hidden_layer_dimensions})
        config.update({'activation': self.activation})
        config.update({'flag_bn': self.flag_bn})
        return config

    # На самом деле нет необходимости определять `from_config` здесь, поскольку 
    # возвращение `cls(**config)` - поведение по умолчанию.
    @classmethod
    def from_config(cls, config):
        return cls(**config)


class Decoder(Layer):
    """
    Класс для decoder модели RSRAE. Отображает вектор z_rsr, полученный в результате
    кодирования исходных данных в вектор z, и последующим отображением вектора z при
    помощи RSR layer (x -> z -> z_rsr), обратно в пространство исходных данных 
    (z_rsr -> x_tilde).
    """

    def __init__(self,
                 inputs_dim,
                 hidden_layer_dimensions,
                 activation,
                 flag_bn=True, 
                 name="Decoder",
                 **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.hidden_layer_dimensions = hidden_layer_dimensions
        self.activation = activation
        self.flag_bn = flag_bn
        self.dense2 = Dense(hidden_layer_dimensions[2], activation=activation,
                            name='decoder_2')
        self.dense1 = Dense(hidden_layer_dimensions[1], activation=activation,
                            name='decoder_1')
        self.dense0 = Dense(hidden_layer_dimensions[0], activation=activation,
                            name='decoder_0')
        self.dense_output = Dense(inputs_dim, activation=activation,
                            name='decoder_output')
        if flag_bn:
            self.batch_normalization2 = BatchNormalization(name="decoder_bn_layer_2")
            self.batch_normalization1 = BatchNormalization(name="decoder_bn_layer_1")
            self.batch_normalization0 = BatchNormalization(name="decoder_bn_layer_0")
    def call(self, inputs):
        """
        Отображние z_rsr -> x_tilde, где x_tilde - вектор, лежащий в пространстве
        исходных даных.
        """
        z_rsr = inputs
        z_rsr = self.dense2(z_rsr)
        if self.flag_bn:
            z_rsr = self.batch_normalization2(z_rsr)
        z_rsr = Dropout(0.3)((z_rsr))
        z_rsr = self.dense1(z_rsr)
        if self.flag_bn:
            z_rsr = self.batch_normalization1(z_rsr)
        z_rsr = Dropout(0.4)((z_rsr))
        z_rsr = self.dense0(z_rsr)
        if self.flag_bn:
            z_rsr = self.batch_normalization0(z_rsr)
        z_rsr = Dropout(0.4)((z_rsr))
        x_tilde = self.dense_output(z_rsr)
        return x_tilde
    
    # Опционально, пользовательский слой может быть сериализован реализацией метода 
    # get_config и метода класса (@classmethod) from_config.
    def get_config(self):
        config = super(Layer, self).get_config()
        config.update({'hidden_layer_dimensions': self.hidden_layer_dimensions})
        config.update({'activation': self.activation})
        config.update({'flag_bn': self.flag_bn})
        return config

    # На самом деле нет необходимости определять `from_config` здесь, поскольку 
    # возвращение `cls(**config)` - поведение по умолчанию.
    @classmethod
    def from_config(cls, config):
        return cls(**config)


class RSRAE(Model):
    """
    Нейросетевая модель-автоэнкодер для обнаружения аномалий с робастным слоем,
    восстанавливающим подпространство (RSR layer между encoder и decoder).
    Комбинируем encoder + RSR layer + decoder в end-to-end модель.
    """

    def __init__(self,
                 inputs_dim, # размерность вектора признаков
                 hidden_layer_dimensions,
                 intrinsic_size, # разерность z_rsr после RSR layer
                 activation,
                 hub_layer,
                 flag_loss,
                 flag_bn=True,
                 flag_normalize=True,
                 learning_rate=1e-3,
                 beta=1,
                 eta=1,
                 t_step=0,
                 ae_loss_norm_type='MSE',
                 rsr_loss_norm_type='MSE',
                 name='RSRAE',
                 **kwargs):
        super(RSRAE, self).__init__(name=name, **kwargs)
        self.inputs_dim = inputs_dim
        self.hidden_layer_dimensions = hidden_layer_dimensions
        self.intrinsic_size = intrinsic_size
        self.activation = activation
        self.flag_bn = flag_bn
        self.flag_normalize = flag_normalize
        self.flag_loss = flag_loss
        self.learning_rate = learning_rate
        self.beta = tf.Variable(beta, dtype=tf.float64, trainable=False)
        self.beta0 = tf.Variable(beta, dtype=tf.float64, trainable=False)
        self.eta = tf.Variable(eta, dtype=tf.float64, trainable=False)
        self.eta0 = tf.Variable(eta, dtype=tf.float64, trainable=False)
        self.t_step = tf.Variable(t_step, dtype=tf.float64, trainable=False)
        self.ae_loss_norm_type = ae_loss_norm_type
        self.rsr_loss_norm_type = rsr_loss_norm_type
        # Для вычисления среднего loss по loss всех батчей в эпохе
        self.loss_tracker = metrics.Mean(name="loss")
        self.auc_tracker = metrics.Mean(name="auc")
        self.ap_tracker = metrics.Mean(name="ap")

        # Создание экземпляров оптимизаторов
        self.optimizer_ae = optimizers.Adam(learning_rate=learning_rate)
        self.optimizer_rsr1 = optimizers.Adam(learning_rate=5 * learning_rate)
        self.optimizer_rsr2 = optimizers.Adam(learning_rate=5 * learning_rate)

        # Слои
        self.emnedding_layer = hub_layer
        self.encoder = Encoder(hidden_layer_dimensions=hidden_layer_dimensions,
                               activation=activation,
                               flag_bn=flag_bn)
        self.rsr = RSR(intrinsic_size=intrinsic_size)
        if flag_normalize:
            self.l2normalization = L2Normalization()
        self.decoder = Decoder(inputs_dim=inputs_dim,
                               hidden_layer_dimensions=hidden_layer_dimensions,
                               activation=activation,
                               flag_bn=flag_bn)
        
    def call(self, inputs):
        e = self.emnedding_layer(inputs)
        e = tf.cast(e, dtype=tf.float64)
        z = self.encoder(e)
        z_rsr = self.rsr(z)
        if self.flag_normalize:
            z_rsr = self.l2normalization(z_rsr)
        x_tilde = self.decoder(z_rsr)
        return e, z, z_rsr, x_tilde

    def ae_loss(self, x, x_tilde):
        """Функция потерь реконструкции автоэнкодера - L_AE."""

        x = tf.reshape(x, (tf.shape(x)[0], -1))
        x_tilde = tf.reshape(x_tilde, (tf.shape(x_tilde)[0], -1))

        # axis=1 для tf.norm => вычисление вдоль оси признаков
        # tf.math.reduce_mean без параметров - mean от элементов матрицы
        if self.ae_loss_norm_type in ['MSE', 'mse', 'Frob', 'F']:
            return tf.math.reduce_mean(tf.math.square(tf.norm(x-x_tilde, 
                                                              ord=2, axis=1)))
        elif self.ae_loss_norm_type in ['L1', 'l1']:
            return tf.math.reduce_mean(tf.norm(x-x_tilde, ord=1, axis=1))
        elif self.ae_loss_norm_type in ['LAD', 'lad', 'L21', 'l21', 'L2', 'l2']:
            return tf.math.reduce_mean(tf.norm(x-x_tilde, ord=2, axis=1))
        else:
            raise Exception("Norm type error!")
    
    def rsr1_loss(self, z, z_rsr, beta, eta):
        """Функция потери для RSR layer - L_RSR1."""
        z_rsr = tf.matmul(z_rsr, tf.transpose(self.rsr.A))
        # z_rsr_new = tf.matmul(z_rsr, self.)

        if self.rsr_loss_norm_type in ['MSE', 'mse', 'Frob', 'F']:
            return tf.math.reduce_mean(tf.math.square(tf.norm(z-z_rsr, ord=2, 
                                                            axis=1)))
        elif self.rsr_loss_norm_type in ['L1', 'l1']:
            return tf.math.reduce_mean(tf.norm(z-z_rsr, ord=1, axis=1))
        elif self.rsr_loss_norm_type in ['LAD', 'lad', 'L21', 'l21', 'L2', 'l2']:
            return tf.math.reduce_mean(tf.norm(z-z_rsr, ord=2, axis=1))
        else:
            raise Exception("Norm type error!")
    
    def rsr2_loss(self):
        """Функция потери для RSR layer - L_RSR2."""
        A = self.rsr.A
        A_T = tf.transpose(A)
        I = tf.eye(self.intrinsic_size, dtype=tf.float64)
        return tf.math.reduce_mean(tf.math.square(tf.linalg.matmul(A_T, A) - I))

    def rsr3_loss(self, z, z_rsr, beta, eta):
        """
        Cтатьи 'Robust principal component analysis by 
        self-organizing rules basedon statistical physics approach', на которую
        ссылается http://files.is.tue.mpg.de/black/papers/delatorreIJCV03.pdf
        """
        z_rsr = tf.matmul(z_rsr, tf.transpose(self.rsr.A))  # AA'z
        e_pca = tf.math.square(tf.norm(z-z_rsr, ord=2, axis=1))
        # self.min_div = tf.reduce_min(e_pca)
        # self.max_div = tf.reduce_max(e_pca)
        # self.mean_div = tf.reduce_mean(e_pca)
        loss = -1 * tf.math.reduce_mean(tf.math.log(1 + tf.math.exp(-beta * e_pca - eta))) / beta
        return loss
        
    def gradients(model, inputs, targets):
        with tf.GradientTape() as tape:
            loss_value = loss_fn(model, inputs, targets)
        return tape.gradient(loss_value, model.trainable_variables)
    
    @tf.function()
    def train_step(self, data):
        """
        Override the method. Будет вызываться при 'model.fit()'.
        Один шаг обучения, на котором вычисляются функции потерь для автоэнкодера и
        RSR layer, и в соотвествии с ними обновляются значения обучаемых переменных - 
        весов нейросети и матрицы A соответсвенно. Будет вызываться от одного батча.
        Заметим, что в этом методе мы используем пользовательские оптимизаторы и функции
        потерь, поэтому перед тренировкой метод compile вызывать не придется.
        """


        x, y = data

        # tf.GradientTape() - записывает операции для автоматического дифференцирования

        # По умолчанию persistent=False и удерживаемые GradientTape, высвобождаются,
        # как только вызывается метод GradientTape.gradient(). Чтобы вычислить несколько
        # градиентов за одно вычисление, требуется задать persistent=true. Это позволяет
        # многократно вызывать метод gradient(), тогда требуется самостоятельно
        # освободить ресурсы с помощью 'del tape'.

        # watch_accessed_variables=True => автоматическое отслеживание всех обучаемых
        # переменные, к которым осуществляется доступ. Так градиенты могут быть
        # запрошены c любого вычисленного результата в tape.
        with tf.GradientTape(persistent=True, watch_accessed_variables=True) as tape:
            # Здесь требуется запустить прямой проход нейросети. Операции применяемые
            # при проходе к входных данным будут записаны на GradientTape. 
            e, z, z_rsr, x_tilde = self.call(x) # прямой проход RSRAE
            z = tf.keras.layers.Flatten()(z) # вроде для текстовых данных необязательно
            # Вычисляем значения функций потерь для этого прохода
            loss_ae = self.ae_loss(e, x_tilde)
            if (self.flag_loss == 0):
                loss_rsr3 = self.rsr1_loss(z, z_rsr, self.beta, self.eta)
            else:
                loss_rsr3 = self.rsr3_loss(z, z_rsr, self.beta, self.eta)
            loss_rsr2 = self.rsr2_loss()
  
        # Метод gradient вычисляет градиенты обучаемых параметров(весов) для минимизации
        # функции потерь, используя операции, записанные в контексте этого tape.
        gradients_ae = tape.gradient(loss_ae, self.trainable_weights)
        gradients_rsr3 = tape.gradient(loss_rsr3, self.rsr.A)
        gradients_rsr2 = tape.gradient(loss_rsr2, self.rsr.A)

        # Обновим значения обучаемых переменных - градиентный шаг чтобы min loss.
        self.optimizer_ae.apply_gradients(grads_and_vars=
                                          zip(gradients_ae, self.trainable_weights))
        self.optimizer_rsr1.apply_gradients(grads_and_vars=
                                            zip([gradients_rsr3], [self.rsr.A]))
        self.optimizer_rsr2.apply_gradients(grads_and_vars=
                                            zip([gradients_rsr2], [self.rsr.A]))
        
        self.loss_tracker.update_state(loss_ae) # обновляем средний loss по батчам

        self.t_step.assign_add(1, use_locking=True)
        self.beta.assign(self.beta0.value() * tf.math.log(self.t_step.value() + 3))
        self.eta.assign(self.eta0.value() * self.t_step.value())

        # Обновляем метрики
        if len(tf.unique(y)[0]) == 2:
            # иначе roc_auc_score бросит ValueError и обучение приостановится
            auc = self.auc_metric(y, cosine_similarity(x_tilde, e))
            self.auc_tracker.update_state(auc)

        ap = self.ap_metric(y, cosine_similarity(x_tilde, e))
        self.ap_tracker.update_state(ap)

        del tape # persistent=True => требуется самостоятельно освободить ресурсы
        return {"loss": self.loss_tracker.result(),
                "auc": self.auc_tracker.result(),
                "ap": self.ap_tracker.result(),
                # "mean_div": self.mean_div,
                # "min_div": self.min_div, 
                # "max_div": self.max_div,
                "beta": self.beta,
                "eta": self.eta,
                "t_step": self.t_step}

    @property
    def metrics(self):
        """
        В пару к train_step. Сбрасывает метрики (`reset_states()`) в начале каждой
        эпохи обучения с помощью 'fit()'. Без этого свойства 'result()' будет 
        возвращать среднее значение с начала обучения.
        """
        return [self.loss_tracker, self.auc_tracker, self.ap_tracker]

    def auc_metric(self, y_true, y_pred):
        return tf.py_function(roc_auc_score, (y_true, y_pred), tf.float64)

    def ap_metric(self, y_true, y_pred):
        return tf.py_function(average_precision_score, (y_true, y_pred), tf.float64)

Tensorflow version = 2.4.1


## Загрузка данных

In [4]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
from sklearn.utils import shuffle
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

c = 0.1  # отношение количества аномальных экземпляров к нормальным

# normal_categories = ['comp.graphics',             
#                      'rec.autos',
#                      'sci.electronics',
#                      'sci.med',
#                      'sci.space',
#                      'sci.space',
#                      'soc.religion.christian',
#                      'soc.religion.christian',
#                      'talk.politics.mideast',
#                      'soc.religion.christian',
#                      'sci.electronics',]

# anomal_categories = ['soc.religion.christian',
#                      'talk.politics.mideast',
#                      'soc.religion.christian',
#                      'talk.politics.mideast',
#                      'rec.sport.hockey',
#                      'soc.religion.christian',
#                      'comp.graphics',
#                      'talk.politics.mideast',
#                      'rec.autos',
#                      'sci.electronics',
#                      'talk.politics.mideast',]

categories = ["comp.graphics",
              "rec.autos",
              "rec.sport.hockey",
              "sci.electronics",
              "sci.med",
              "sci.space",
              'misc.forsale',
              'talk.religion.misc']

experimant_cnt = 0
all_experiments = len(categories) * (len(categories) -  1)   #len(normal_categories)
auc_list0 = []
auc_list1 = []

# Формирование словаря с категориями
dataset = {}
for cat in categories:
    dataset[cat] = fetch_20newsgroups(subset='all', categories=[cat],
                            shuffle=True, random_state=123,
                            remove=('headers', 'footers'), return_X_y=True)[0]

# Перебираем пары категорий

for i in range(len(categories)):
    for j in range(len(categories)):
        if i == j:
            continue
        c1 = categories[i]
        c2 = categories[j]

        experimant_cnt += 1

        # Формирование нормальной и аномальной выборок
        normal_data = dataset[c1]
        anomal_data = dataset[c2][:min(int(c * len(normal_data)) + 1, len(dataset[c2]))]
        all_data = normal_data + anomal_data
        x = pd.Series(all_data)
        y = np.array([False] * len(normal_data) + [True] * len(anomal_data))
        all_data, x, y = shuffle(all_data, x, y, random_state=123)

        # Изначальный слой USE
        layer = hub_layer

        config = layer.get_config()
        weights = layer.get_weights()
        cloned_layer = type(layer).from_config(config)
        cloned_layer.set_weights(weights)

        hub_layer_now = cloned_layer


        # Тренировка модели
        model_rsrae = RSRAE(inputs_dim=512,
                            hidden_layer_dimensions=[512, 1024, 2048],
                            intrinsic_size=20,
                            activation='relu',
                            hub_layer=hub_layer_now,
                            flag_loss=1,
                            learning_rate=1e-4,
                            beta=1.0,
                            eta=0.0015,
                            ae_loss_norm_type='MSE',
                            rsr_loss_norm_type='MSE',)
        model_rsrae.compile(run_eagerly=True)
        model_rsrae.fit(x, y,
                        batch_size=512,
                        epochs=10,
                        verbose=0)
        
        e, _, _, x_predict = model_rsrae.call(x)
        auc1 = roc_auc_score(y, cosine_similarity(x_predict, e))
        auc_list1.append(auc1)

        # Тренировка модели
        model_rsrae = RSRAE(inputs_dim=512,
                            hidden_layer_dimensions=[512, 1024, 2048],
                            intrinsic_size=20,
                            activation='relu',
                            hub_layer=hub_layer_now,
                            flag_loss=0,
                            learning_rate=1e-4,
                            beta=1.0,
                            eta=0.0015,
                            ae_loss_norm_type='MSE',
                            rsr_loss_norm_type='MSE',)
        model_rsrae.compile(run_eagerly=True)
        model_rsrae.fit(x, y,
                        batch_size=512,
                        epochs=10,
                        verbose=0)
        
        e, _, _, x_predict = model_rsrae.call(x)
        auc = roc_auc_score(y, cosine_similarity(x_predict, e))
        auc_list0.append(auc)

        print("-" * 50)
        print("Эксперимент №{}/{}  с normal = {}, anomal = {}".format(
            experimant_cnt, all_experiments, c1, c2))
        print("old auc = {}".format(auc))
        print("new auc = {}".format(auc1))
        print("-" * 50)

--------------------------------------------------
Эксперимент №1/56  с normal = comp.graphics, anomal = rec.autos
old auc = 0.7847599471443254
new auc = 0.9433269710762002
--------------------------------------------------
--------------------------------------------------
Эксперимент №2/56  с normal = comp.graphics, anomal = rec.sport.hockey
old auc = 0.9847410701176669
new auc = 0.9890618117750697
--------------------------------------------------


--------------------------------------------------
Эксперимент №3/56  с normal = comp.graphics, anomal = sci.electronics
old auc = 0.6719067894372549
new auc = 0.702340751305661
--------------------------------------------------


--------------------------------------------------
Эксперимент №4/56  с normal = comp.graphics, anomal = sci.med
old auc = 0.8987771881620068
new auc = 0.9359859051534283
--------------------------------------------------


--------------------------------------------------
Эксперимент №5/56  с normal = comp.graphics, anomal = sci.space
old auc = 0.7867420349434738
new auc = 0.8586425320385092
--------------------------------------------------


--------------------------------------------------
Эксперимент №6/56  с normal = comp.graphics, anomal = misc.forsale
old auc = 0.8343121421230362
new auc = 0.8966902279925331
--------------------------------------------------


...

--------------------------------------------------
Эксперимент №52/56  с normal = talk.religion.misc, anomal = rec.sport.hockey
old auc = 0.963780204226064
new auc = 0.9593064401981599
--------------------------------------------------


--------------------------------------------------
Эксперимент №53/56  с normal = talk.religion.misc, anomal = sci.electronics
old auc = 0.8672025073298959
new auc = 0.9654989384288747
--------------------------------------------------


--------------------------------------------------
Эксперимент №54/56  с normal = talk.religion.misc, anomal = sci.med
old auc = 0.7491659084015772
new auc = 0.859316550399353
--------------------------------------------------


--------------------------------------------------
Эксперимент №55/56  с normal = talk.religion.misc, anomal = sci.space
old auc = 0.8576989182084722
new auc = 0.9294307956728339
--------------------------------------------------


--------------------------------------------------
Эксперимент №56/56  с normal = talk.religion.misc, anomal = misc.forsale
old auc = 0.9461884541502376
new auc = 0.9205843696289555
--------------------------------------------------


In [5]:
auc_np = np.array(auc_list0)
print("*" * 50)
print("OLD Медиана auc = {}".format(np.median(auc_np)))
print("OLD Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)
print("Эксперименты завершены!")

auc_np = np.array(auc_list1)
print("*" * 50)
print("NEW Медиана auc = {}".format(np.median(auc_np)))
print("NEW Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)
print("Эксперименты завершены!")

**************************************************
OLD Медиана auc = 0.8241060606060606
OLD Среднее auc = 0.8009078631764287
**************************************************
Эксперименты завершены!
**************************************************
NEW Медиана auc = 0.9326060572164381
NEW Среднее auc = 0.8829758033224971
**************************************************
Эксперименты завершены!


$w_1, w_2, ..., w_n$

$v_1, v_2, ..., v_n$

${\sum\limits^{N}}{\sum\limits_{i}^n}{\sum\limits_{j}^n}cos(v_i, w_i)$

$\underset{A}{\text{minimize}} ||z-zAA^T||\quad\text{if}\ \  AA^T = I$

$L = {\sum\limits_{i=1}^N\big|\big|z^{(i)} - z^{(i)}AA^T\big|\big|} + ||AA^T - I||$

$L = {\sum\limits_{i=1}^N\Big(V_i\big|\big|z^{(i)} - z^{(i)}AA^T\big|\big|} + \eta(1 - V_i)\Big)$

$L' = -\frac{1}{\beta}\sum\limits_{i=1}^Nlog\big(1 + e^{-\beta\  \cdot\  (||z^{(i)} - z^{(i)}AA^T|| - \eta)}\big)$ 

$L = L' + ||AA^T - I||$